In [1]:
import pandas as pd
pd.set_option('display.max_columns', 500)
import os

In [2]:
stack_overflow_files = (os.listdir("data/"))
# not worth going from 2011-2014. No data scientists.
# ok, so decision to do (2019 maybe) 2020-2023 for analysis

# these are the common columns from 2019-2023
common_cols = [
    'Age',
 'CompTotal',
 'Country',
 'DevType',
 'EdLevel',
 'Employment',
 'MainBranch',
 'OrgSize',
 'SOAccount',
 'SOComm',
 'SOPartFreq',
 'SOVisitFreq',
 'SurveyEase',
 'SurveyLength',
 'Year',
 'YearsCode',
 'YearsCodePro'
]

# salary cols to show salary trends
salary_cols = [
    'LanguageWorkedWith', 
]

# show skill trends between the years
skills_cols = [
    
]

stack_overflow_files

['survey_results_public_2019.csv',
 'survey_results_public_2020.csv',
 'survey_results_public_2021.csv',
 'survey_results_public_2022.csv',
 'survey_results_public_2023.csv']

In [3]:
# # this is all testing for the below function
# df_2023 = pd.read_csv(f"data/{stack_overflow_files[-1]}", encoding='ISO-8859-1')
# # df_2023.columns
# df_2023.dropna(inplace=True, subset=["DevType", "ConvertedCompYearly"])
# df_data_2023 = df_2023[df_2023["DevType"].str.contains("data", case=False)]

# df_2022 = pd.read_csv(f"data/{stack_overflow_files[-2]}", encoding='ISO-8859-1')
# # df_2022.columns
# df_2022.dropna(inplace=True, subset=["DevType", "ConvertedCompYearly"])
# df_data_2022 = df_2022[df_2022["DevType"].str.contains("data", case=False)]

# df_2021 = pd.read_csv(f"data/{stack_overflow_files[-3]}", encoding='ISO-8859-1')
# # df_2021.columns
# df_2021.dropna(inplace=True, subset=["DevType", "ConvertedCompYearly"])
# # df_2021["DevType"].value_counts()
# df_data_2021 = df_2021[df_2021["DevType"].str.contains("data", case=False)]

# df_2020 = pd.read_csv(f"data/{stack_overflow_files[-4]}", encoding='ISO-8859-1')
# df_2020.dropna(inplace=True, subset=["DevType", "ConvertedComp"])
# # df_2020["DevType"].value_counts()
# df_data_2020 = df_2020[df_2020["DevType"].str.contains("data", case=False)]

# df_2019 = pd.read_csv(f"data/{stack_overflow_files[-5]}", encoding='ISO-8859-1')
# # df_2019.columns
# df_2019.dropna(inplace=True, subset=["DevType", "ConvertedComp"])
# # df_2019["DevType"].value_counts()
# df_data_2019 = df_2019[df_2019["DevType"].str.contains("data", case=False)]

In [81]:
def read_stackoverflow_csvs() -> dict:
    """
    Reads CSVs and gets the numbe of data professionals. Any empty values are dropped from job title and 
    salary so we will always have data. Other columns may have nans.
    Data Manipulation:
    - dropping nans from salary and devtype combined
    - Changing the salary column to ConvertedCompYearly so we can merge all data frames comes time
    - Converting specific columns that mean the same thing per year into a singular name
    """
    frames = {}
    stack_o_files = os.listdir("data/")
    for file in stack_o_files:
        year = file[-8:-4]
        df = pd.read_csv(f"data/{file}", encoding='ISO-8859-1')

        # standardize compensation columns
        if 'ConvertedComp' in df.columns:
            df = df.rename(columns={'ConvertedComp': 'ConvertedCompYearly'})

        # standardize some columns
        standard = ["Language", "Database", "Platform", "WebFrame", "MiscTech"]
        for stan in standard:
            if f"{stan}WorkedWith" in df.columns:
                df = df.rename(columns={f'{stan}WorkedWith': f'{stan}HaveWorkedWith', f'{stan}DesireNextYear':f'{stan}WantToWorkWith'})

        # long, bad version of the above
        # if "LanguageWorkedWith" in df.columns:
        #     df = df.rename(columns={'LanguageWorkedWith': 'LanguageHaveWorkedWith', 'LanguageDesireNextYear':'LanguageWantToWorkWith'})

        # if "DatabaseWorkedWith" in df.columns:
        #     df = df.rename(columns={'DatabaseWorkedWith': 'DatabaseHaveWorkedWith', 'DatabaseDesireNextYear':'DatabaseWantToWorkWith'})

        # if "PlatformWorkedWith" in df.columns:
        #     df = df.rename(columns={'PlatformWorkedWith': 'PlatformHaveWorkedWith', 'PlatformDesireNextYear':'PlatformWantToWorkWith'})

        # if "WebFrameWorkedWith" in df.columns:
        #     df = df.rename(columns={'WebFrameWorkedWith': 'WebFrameHaveWorkedWith', 'WebFrameDesireNextYear':'WebFrameWantToWorkWith'})

        # if "MiscTechWorkedWith" in df.columns:
        #     df = df.rename(columns={'MiscTechWorkedWith': 'MiscTechHaveWorkedWith', 'MiscTechDesireNextYear':'MiscTechWantToWorkWith'})
        
        df = df.dropna(subset=["DevType", "ConvertedCompYearly"])
        df = df[df["DevType"].str.contains("data", case=False)]
        df["Year"] = [year] * len(df)
        frames[f"df_data_{year}"] = df
    return frames

In [82]:
frames_dict = read_stackoverflow_csvs()

In [83]:
# this is the number of entries we are working with in our frames
# seeing how to standardize the columns some more

query = "Eth"
for key, frame in frames_dict.items():
    lang = []
    for col in frame.columns:
        lang.append(col) if query in col else None
    print(f"{key}\t{len(frame)}\t{lang}")

df_data_2019	13393	['Ethnicity']
df_data_2020	8294	['Ethnicity']
df_data_2021	9272	['Ethnicity']
df_data_2022	6921	['Ethnicity']
df_data_2023	2480	[]


In [84]:
print(frames_dict["df_data_2019"].columns)

Index(['Respondent', 'MainBranch', 'Hobbyist', 'OpenSourcer', 'OpenSource',
       'Employment', 'Country', 'Student', 'EdLevel', 'UndergradMajor',
       'EduOther', 'OrgSize', 'DevType', 'YearsCode', 'Age1stCode',
       'YearsCodePro', 'CareerSat', 'JobSat', 'MgrIdiot', 'MgrMoney',
       'MgrWant', 'JobSeek', 'LastHireDate', 'LastInt', 'FizzBuzz',
       'JobFactors', 'ResumeUpdate', 'CurrencySymbol', 'CurrencyDesc',
       'CompTotal', 'CompFreq', 'ConvertedCompYearly', 'WorkWeekHrs',
       'WorkPlan', 'WorkChallenge', 'WorkRemote', 'WorkLoc', 'ImpSyn',
       'CodeRev', 'CodeRevHrs', 'UnitTests', 'PurchaseHow', 'PurchaseWhat',
       'LanguageHaveWorkedWith', 'LanguageWantToWorkWith',
       'DatabaseHaveWorkedWith', 'DatabaseWantToWorkWith',
       'PlatformHaveWorkedWith', 'PlatformWantToWorkWith',
       'WebFrameHaveWorkedWith', 'WebFrameWantToWorkWith',
       'MiscTechHaveWorkedWith', 'MiscTechWantToWorkWith', 'DevEnviron',
       'OpSys', 'Containers', 'BlockchainOrg', 'B

In [85]:
print(frames_dict["df_data_2020"].columns)

Index(['Respondent', 'MainBranch', 'Hobbyist', 'Age', 'Age1stCode', 'CompFreq',
       'CompTotal', 'ConvertedCompYearly', 'Country', 'CurrencyDesc',
       'CurrencySymbol', 'DatabaseWantToWorkWith', 'DatabaseHaveWorkedWith',
       'DevType', 'EdLevel', 'Employment', 'Ethnicity', 'Gender', 'JobFactors',
       'JobSat', 'JobSeek', 'LanguageWantToWorkWith', 'LanguageHaveWorkedWith',
       'MiscTechWantToWorkWith', 'MiscTechHaveWorkedWith',
       'NEWCollabToolsDesireNextYear', 'NEWCollabToolsWorkedWith', 'NEWDevOps',
       'NEWDevOpsImpt', 'NEWEdImpt', 'NEWJobHunt', 'NEWJobHuntResearch',
       'NEWLearn', 'NEWOffTopic', 'NEWOnboardGood', 'NEWOtherComms',
       'NEWOvertime', 'NEWPurchaseResearch', 'NEWPurpleLink', 'NEWSOSites',
       'NEWStuck', 'OpSys', 'OrgSize', 'PlatformWantToWorkWith',
       'PlatformHaveWorkedWith', 'PurchaseWhat', 'Sexuality', 'SOAccount',
       'SOComm', 'SOPartFreq', 'SOVisitFreq', 'SurveyEase', 'SurveyLength',
       'Trans', 'UndergradMajor', 'Webfr

In [86]:
print(frames_dict["df_data_2021"].columns)

Index(['ResponseId', 'MainBranch', 'Employment', 'Country', 'US_State',
       'UK_Country', 'EdLevel', 'Age1stCode', 'LearnCode', 'YearsCode',
       'YearsCodePro', 'DevType', 'OrgSize', 'Currency', 'CompTotal',
       'CompFreq', 'LanguageHaveWorkedWith', 'LanguageWantToWorkWith',
       'DatabaseHaveWorkedWith', 'DatabaseWantToWorkWith',
       'PlatformHaveWorkedWith', 'PlatformWantToWorkWith',
       'WebframeHaveWorkedWith', 'WebframeWantToWorkWith',
       'MiscTechHaveWorkedWith', 'MiscTechWantToWorkWith',
       'ToolsTechHaveWorkedWith', 'ToolsTechWantToWorkWith',
       'NEWCollabToolsHaveWorkedWith', 'NEWCollabToolsWantToWorkWith', 'OpSys',
       'NEWStuck', 'NEWSOSites', 'SOVisitFreq', 'SOAccount', 'SOPartFreq',
       'SOComm', 'NEWOtherComms', 'Age', 'Gender', 'Trans', 'Sexuality',
       'Ethnicity', 'Accessibility', 'MentalHealth', 'SurveyLength',
       'SurveyEase', 'ConvertedCompYearly', 'Year'],
      dtype='object')


In [87]:
print(frames_dict["df_data_2022"].columns)

Index(['ResponseId', 'MainBranch', 'Employment', 'RemoteWork',
       'CodingActivities', 'EdLevel', 'LearnCode', 'LearnCodeOnline',
       'LearnCodeCoursesCert', 'YearsCode', 'YearsCodePro', 'DevType',
       'OrgSize', 'PurchaseInfluence', 'BuyNewTool', 'Country', 'Currency',
       'CompTotal', 'CompFreq', 'LanguageHaveWorkedWith',
       'LanguageWantToWorkWith', 'DatabaseHaveWorkedWith',
       'DatabaseWantToWorkWith', 'PlatformHaveWorkedWith',
       'PlatformWantToWorkWith', 'WebframeHaveWorkedWith',
       'WebframeWantToWorkWith', 'MiscTechHaveWorkedWith',
       'MiscTechWantToWorkWith', 'ToolsTechHaveWorkedWith',
       'ToolsTechWantToWorkWith', 'NEWCollabToolsHaveWorkedWith',
       'NEWCollabToolsWantToWorkWith', 'OpSysProfessional use',
       'OpSysPersonal use', 'VersionControlSystem', 'VCInteraction',
       'VCHostingPersonal use', 'VCHostingProfessional use',
       'OfficeStackAsyncHaveWorkedWith', 'OfficeStackAsyncWantToWorkWith',
       'OfficeStackSyncHaveWork

In [88]:
# do they have similar columns?
def find_similar_col(frames):
    union = []
    for key, frame in frames.items():
        union.append(set(frame.columns))
        
    standard = union[0]
    for cols in union[1:]:
        standard = standard.intersection(cols)
    return standard

In [89]:
find_similar_col(frames_dict)

{'Age',
 'CompTotal',
 'ConvertedCompYearly',
 'Country',
 'DatabaseHaveWorkedWith',
 'DatabaseWantToWorkWith',
 'DevType',
 'EdLevel',
 'Employment',
 'LanguageHaveWorkedWith',
 'LanguageWantToWorkWith',
 'MainBranch',
 'MiscTechHaveWorkedWith',
 'MiscTechWantToWorkWith',
 'OrgSize',
 'PlatformHaveWorkedWith',
 'PlatformWantToWorkWith',
 'SOAccount',
 'SOComm',
 'SOPartFreq',
 'SOVisitFreq',
 'SurveyEase',
 'SurveyLength',
 'Year',
 'YearsCode',
 'YearsCodePro'}

In [90]:
# play around with the number and see if this is the spread that we want
for key, frame in frames_dict.items():
    grouped = (frame.groupby("Country").count())
    length = len(grouped[grouped["MainBranch"] > 10])
    print(f"""{key}: {length}
    max: {grouped['MainBranch'].idxmax()}
    min: {grouped['MainBranch'].idxmin()}""")

df_data_2019: 83
    max: United States
    min: Andorra
df_data_2020: 69
    max: United States
    min: Andorra
df_data_2021: 70
    max: United States of America
    min: Andorra
df_data_2022: 63
    max: United States of America
    min: Algeria
df_data_2023: 36
    max: United States of America
    min: Albania
